In [ ]:
# Cell 1: Setup & Environment Installation

# --- ติดตั้ง Libraries ---
# timm: คลังโมเดล PyTorch ขนาดใหญ่ (Vision Transformers, ConvNeXt, etc.)
# opendatasets: สำหรับดาวน์โหลดข้อมูลจาก Kaggle
# albumentations: สำหรับทำ Data Augmentation
!pip install timm opendatasets albumentations --quiet

# --- Import Libraries ที่จำเป็น ---
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.notebook import tqdm
import timm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

print("Setup Complete. Libraries including 'timm' are ready.")


In [ ]:
# Cell 2: Configuration
class CFG:
    # --- Paths ---
    DATA_DIR = './garbage-classification/'
    
    # --- Model ---
    MODEL_NAME = 'convnext_tiny' # โมเดลจาก timm (ลองเปลี่ยนเป็น 'vit_base_patch16_224')
    PRETRAINED = True
    
    # --- Training Parameters ---
    IMG_SIZE = 224
    BATCH_SIZE = 32
    EPOCHS = 10 # โจทย์จริงอาจใช้ 20-30
    LEARNING_RATE = 1e-4
    
    # --- Environment ---
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    NUM_WORKERS = 2

print(f"Configuration loaded. Using device: {CFG.DEVICE}")


In [ ]:
# Cell 3: Data Acquisition & Preparation
import opendatasets as od

dataset_url = 'https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification'
od.download(dataset_url)

# --- สร้าง DataFrame จากโครงสร้างโฟลเดอร์ ---
image_paths = []
labels = []

# วนลูปในแต่ละโฟลเดอร์ (ซึ่งเป็นชื่อคลาส)
for class_name in os.listdir(CFG.DATA_DIR):
    class_dir = os.path.join(CFG.DATA_DIR, class_name)
    if os.path.isdir(class_dir):
        for img_name in os.listdir(class_dir):
            image_paths.append(os.path.join(class_dir, img_name))
            labels.append(class_name)

df = pd.DataFrame({'image_path': image_paths, 'label': labels})

# --- แปลง Label (ชื่อคลาส) เป็นตัวเลข (Integer) ---
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# เก็บชื่อคลาสและจำนวนคลาสไว้ใน CFG
CFG.CLASS_NAMES = label_encoder.classes_
CFG.NUM_CLASSES = len(CFG.CLASS_NAMES)

print(f"Data prepared. Found {len(df)} images belonging to {CFG.NUM_CLASSES} classes.")
print("Class names:", CFG.CLASS_NAMES)
df.head()


In [ ]:
# Cell 4: Analyze and Handle Class Imbalance

# --- แสดงกราฟการกระจายตัวของข้อมูล ---
plt.figure(figsize=(12, 6))
sns.countplot(y=df['label'], order = df['label'].value_counts().index)
plt.title('Class Distribution')
plt.show()

# --- คำนวณ Weights สำหรับ Sampler ---
class_counts = df['label'].value_counts().sort_index()
class_weights = 1. / torch.tensor(class_counts.values, dtype=torch.float)
sample_weights = class_weights[df['label_encoded'].values]

print("Weights for each sample calculated for WeightedRandomSampler.")


In [ ]:
# Cell 5: Augmentations and Custom Dataset

train_transforms = A.Compose([
    A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.CoarseDropout(max_holes=8, max_height=16, max_width=16, p=0.3),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

valid_transforms = A.Compose([
    A.Resize(CFG.IMG_SIZE, CFG.IMG_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

class GarbageDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        label = row['label_encoded']
        
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            image = self.transforms(image=image)['image']
        
        return image, torch.tensor(label, dtype=torch.long)


In [ ]:
# Cell 6: Create DataLoaders with Sampler

# แบ่งข้อมูล
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# สร้าง Datasets
train_dataset = GarbageDataset(train_df, transforms=train_transforms)
val_dataset = GarbageDataset(val_df, transforms=valid_transforms)

# สร้าง Sampler
sampler = WeightedRandomSampler(
    weights=sample_weights[train_df.index], 
    num_samples=len(train_df), 
    replacement=True
)

# สร้าง DataLoaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=CFG.BATCH_SIZE, 
    sampler=sampler, # <-- ใช้ Sampler ที่นี่
    num_workers=CFG.NUM_WORKERS
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=CFG.BATCH_SIZE, 
    shuffle=False, 
    num_workers=CFG.NUM_WORKERS
)

print(f"DataLoaders created. Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")


In [ ]:
# Cell 7: Define Model, Loss, and Optimizer

# สร้างโมเดลจาก timm
model = timm.create_model(
    CFG.MODEL_NAME,
    pretrained=CFG.PRETRAINED,
    num_classes=CFG.NUM_CLASSES
).to(CFG.DEVICE)

# กำหนด Loss Function และ Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.LEARNING_RATE)

print(f"Model '{CFG.MODEL_NAME}' created with {CFG.NUM_CLASSES} output classes.")
# print(model) # Uncomment to see the model architecture


In [ ]:
# Cell 8: The Training Loop

best_val_acc = 0.0
BEST_MODEL_PATH = f"{CFG.MODEL_NAME}_best.pth"

for epoch in range(CFG.EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{CFG.EPOCHS} ---")
    
    # --- Training Phase ---
    model.train()
    train_loss, train_correct = 0, 0
    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(CFG.DEVICE), labels.to(CFG.DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        train_correct += torch.sum(preds == labels.data)

    # --- Validation Phase ---
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(CFG.DEVICE), labels.to(CFG.DEVICE)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data)
            
    # คำนวณค่าเฉลี่ย
    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_acc = train_correct.double() / len(train_loader.sampler)
    
    avg_val_loss = val_loss / len(val_loader.dataset)
    avg_val_acc = val_correct.double() / len(val_loader.dataset)
    
    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f} | Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}")

    # บันทึกโมเดลที่ดีที่สุด
    if avg_val_acc > best_val_acc:
        print(f"Validation accuracy improved from {best_val_acc:.4f} to {avg_val_acc:.4f}. Saving model...")
        best_val_acc = avg_val_acc
        torch.save(model.state_dict(), BEST_MODEL_PATH)

print("\nFinished Training!")


In [ ]:
# Cell 9: Inference and Submission File Generation

# --- สร้าง Test Loader (ในที่นี้เราใช้ validation set เป็นตัวจำลอง) ---
test_df = val_df.copy().reset_index(drop=True)
test_df['image_id'] = test_df['image_path'].apply(lambda x: os.path.basename(x))

test_dataset = GarbageDataset(test_df, transforms=valid_transforms)
test_loader = DataLoader(test_dataset, batch_size=CFG.BATCH_SIZE, shuffle=False)

# --- โหลดโมเดลที่ดีที่สุดและทำนายผล ---
model.load_state_dict(torch.load(BEST_MODEL_PATH))
model.eval()

all_preds = []
with torch.no_grad():
    for images, _ in tqdm(test_loader, desc="Generating Predictions"):
        images = images.to(CFG.DEVICE)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

# แปลง index กลับเป็นชื่อคลาส
predicted_labels = label_encoder.inverse_transform(all_preds)

# --- สร้าง DataFrame สำหรับ Submission ---
submission_df = pd.DataFrame({
    'image_id': test_df['image_id'],
    'label': predicted_labels
})
submission_df.to_csv('submission.csv', index=False)

print("\nsubmission.csv created successfully!")
display(submission_df.head())
